<a href="https://colab.research.google.com/github/hail-members/llm-based-services/blob/main/Chapter_5_%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Train 데이터 다운로드
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json

# Dev 데이터 다운로드
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

import json

# Train 데이터 로드
with open("KorQuAD_v1.0_train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Dev 데이터 로드
with open("KorQuAD_v1.0_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# 데이터 구조 확인
print("Train Data Keys:", train_data.keys())
print("Example Data:", train_data["data"][0])  # 첫 번째 문단 출력

--2025-04-06 06:45:08--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   130MB/s    in 0.3s    

2025-04-06 06:45:11 (130 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2025-04-06 06:45:11--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [2]:
# 라이브러리 임포트
import torch
from transformers import GPT2LMHeadModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader


# KoGPT2 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)
tokenizer.add_special_tokens({'additional_special_tokens': ['<usr>', '<sys>']})
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [3]:
# 데이터셋 정의 (질문과 답변을 GPT2 입력 형식으로 변환)
class KorQuADDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=30):
        self.data = data["data"]
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.total_qas = []  # 모든 질문-답변 쌍을 저장

        # 전체 질문-답변 쌍을 리스트로 저장
        for article in self.data:
            for paragraph in article["paragraphs"]:
                for qa in paragraph["qas"]:
                    self.total_qas.append((paragraph["context"], qa))

    def __len__(self):
        return len(self.total_qas)

    def __getitem__(self, idx):
        context, qa = self.total_qas[idx]
        question = qa["question"]
        answer = qa["answers"][0]["text"]

        # 전체 시퀀스 생성 (질문+답변)
        full_text = f"<usr> {question} </s> <sys> {answer} </s>"

        # 통합 토큰화
        encoding = self.tokenizer(
            full_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        # 레이블 마스킹 처리
        input_ids = encoding.input_ids.squeeze()

        labels = torch.full(input_ids.shape, fill_value=self.tokenizer.pad_token_id)  # 패딩 값으로 채운 텐서
        labels[:-1] = input_ids[1:]  # 두 번째부터 마지막까지

        return {
            "input_ids": input_ids,
            "labels": labels
        }


train_dataset = KorQuADDataset(train_data, tokenizer)
dev_dataset = KorQuADDataset(dev_data, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=4)


In [4]:
# 저장된 데이터 보기
for batch in train_dataloader:
    print(batch['input_ids'][0])
    print(batch['input_ids'].shape)
    print(batch['labels'][0])
    print(batch['labels'].shape)
    # print(tokenizer.decode(batch["input_ids"][0], skip_special_tokens=False))
    break

tensor([    2, 25565,  9772,   420, 37082,   424,   431, 18258,  8143, 12925,
         7182,  9024, 23127, 12743,  9774,   739,  9565, 11084,  9469,  9522,
        13573,  9168,  9580,  9025,  9341,  9024,  7281,  7609, 10062, 10319])
torch.Size([4, 30])
tensor([25565,  9772,   420, 37082,   424,   431, 18258,  8143, 12925,  7182,
         9024, 23127, 12743,  9774,   739,  9565, 11084,  9469,  9522, 13573,
         9168,  9580,  9025,  9341,  9024,  7281,  7609, 10062, 10319,     3])
torch.Size([4, 30])


In [5]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 문장 생성 함수 정의
def generate_sentence(model, seed_text, max_length=50):
    input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)
    gen_ids = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        use_cache=True,
    )
    generated_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    return generated_text

question = "인공지능이란?"
context = ""

# 입력 텍스트 생성
input_text = f"<usr> {question} <sys> {context} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=100,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=150,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print(f"질문: {question}")
print(f"컨텍스트: {context}")
print(f"생성된 답변: {generated_text}")


model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

질문: 인공지능이란?
컨텍스트: 
생성된 답변: 인공지능이란?   
, , . (중략) 이번에 출시된 신제품은 '스마트폰용 스마트패드'다.
이 제품은 스마트폰을 통해 다양한 콘텐츠를 즐길 수 있는 것이 특징이다.
특히 기존 제품보다 최대 2배 이상 빠른 속도로 데이터를 전송


In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-3)

# 학습 루프 정의 (tqdm으로 Progress Bar 추가)
epochs = 1
model.train()

# 조기 종료 조건 설정
# max_batches_per_epoch = 1000  # 한 에포크에서 최대 실행할 배치 수

for epoch in range(epochs):
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}", total=len(train_dataloader))

    for batch_idx, batch in progress_bar:
        # if batch_idx >= max_batches_per_epoch:  # 조기 종료 조건
        #     print(f"Stopping early at batch {batch_idx} in epoch {epoch + 1}")
        #     break

        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        print(f"질문: {tokenizer.decode(input_ids[0], skip_special_tokens=False)}")

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"Batch Loss": loss.item()})

    print(f"Epoch {epoch + 1} completed. Average Loss: {epoch_loss / (batch_idx + 1)}")

# 모델 저장
model.save_pretrained("./kogpt2-korquad-finetuned")
tokenizer.save_pretrained("./kogpt2-korquad-finetuned")


Epoch 1:   0%|          | 0/15102 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


질문: <usr> 미국은 고립주의를 언제까지 취했나?  </s> <sys> 19세기 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>



Epoch 1:   0%|          | 2/15102 [00:01<2:12:11,  1.90it/s, Batch Loss=10.5]

질문: <usr> EXO의 2016년 리패키지 앨범이름은? </s> <sys> 《LOTTO》 </s><pad><pad><pad>
질문: <usr> 에나쓰 유타카가 닛폰햄에서 이적한 팀은? </s> <sys> 세이부 라이온스 </s><pad><pad><pad><pad>



Epoch 1:   0%|          | 4/15102 [00:01<1:04:58,  3.87it/s, Batch Loss=14.8]

...


Epoch 1:   3%|▎         | 435/15102 [00:40<23:24, 10.44it/s, Batch Loss=5.49]

질문: <usr> 원소를 공격해 여러 차례 패배하면서도 싸움을 거듭하여 주위를 피폐하게 하게 하였던 관료의 이름은? </s> <sys> 공손찬 </s><pad><pad><pad>
질문: <usr> 두개의 흉갑기병 사단을 불러오게 한 군대는 무엇인가? </s> <sys> 카파레리의 군대 </s><pad><pad><pad><pad>



Epoch 1:   3%|▎         | 437/15102 [00:40<24:07, 10.13it/s, Batch Loss=5.15]

질문: <usr> 스타 트렉은 1987년에서 1994년까지 총 몇 시즌을 방영하였는 가? </s> <sys> 7 시즌 </s><pad><pad><pad><pad>
질문: <usr> 2005년 열린우리당이 재도입을 시도한 정책은? </s> <sys> 토지공개념 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 멩겔레의 아내 이름은? </s> <sys> 이레네 쇤바인 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>



Epoch 1:   3%|▎         | 437/15102 [00:41<24:07, 10.13it/s, Batch Loss=5.99]

In [ ]:

question = "인공지능이란?"

# 입력 텍스트 생성
input_text = f"<usr> {question} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=100,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=400,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

# 결과 출력
print(f"질문: {question}")
print(f"생성된 답변: {generated_text}")

In [ ]:
model.eval()
for epoch in range(epochs):
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}", total=len(train_dataloader))

    for batch_idx, batch in progress_bar:
        # if batch_idx >= max_batches_per_epoch:  # 조기 종료 조건
        #     print(f"Stopping early at batch {batch_idx} in epoch {epoch + 1}")
        #     break

        input_ids = batch["input_ids"].to(device)
        # cut off where the question endss
        # find where the question ends
        sep_token = tokenizer.convert_tokens_to_ids("<sys>")
        sep_positions = (input_ids == sep_token).nonzero(as_tuple=True)[1]
        if len(sep_positions) >= 1:
            input_ids = input_ids[:,:sep_positions[0]+1]


        labels = batch["labels"].to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                max_length=513,
                repetition_penalty=2.0,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
            )

        print(f"질문: {tokenizer.decode(input_ids[0], skip_special_tokens=False)}")
        print(f"생성된 답변: {tokenizer.decode(output_ids[0], skip_special_tokens=False)}")
        break